**Libraries**

In [13]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

**Data**

In [14]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Converting labels to one-hot encoding
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels,10)

print(train_images.shape)
print(test_images.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


**The Inception Model**

In [15]:
def inception_module(inputs, filters):
    # Convolution with 1x1 filter size
    conv1x1 = layers.Conv2D(filters=filters[0], kernel_size=1, activation='relu', padding='same')(inputs)

    # 1x1 convolution followed by 3x3 convolution
    conv3x3_reduce = layers.Conv2D(filters=filters[1], kernel_size=1, activation='relu', padding='same')(inputs)
    conv3x3 = layers.Conv2D(filters=filters[2], kernel_size=3, activation='relu', padding='same')(conv3x3_reduce)

    # 1x1 convolution followed by 5x5 convolution
    conv5x5_reduce = layers.Conv2D(filters=filters[3], kernel_size=1, activation='relu', padding='same')(inputs)
    conv5x5 = layers.Conv2D(filters=filters[4], kernel_size=5, activation='relu', padding='same')(conv5x5_reduce)

    # Max pooling followed by 1x1 convolution
    maxpool = layers.MaxPooling2D(pool_size=3, strides=1, padding='same')(inputs)
    pool_conv = layers.Conv2D(filters=filters[5], kernel_size=1, activation='relu', padding='same')(maxpool)

    # Concatenating the outputs of all operations along the channel axis
    output = layers.concatenate([conv1x1, conv3x3, conv5x5, pool_conv], axis=-1)
    return output

**Architecture of the Network**

In [16]:
def create_inception_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    # Convolutional layers for feature extraction
    conv1 = layers.Conv2D(64, 7, activation='relu', padding='same', strides=2)(inputs)
    pool1 = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(conv1)

    # Additional Inception modules
    inception1 = inception_module(pool1, filters=[64, 96, 128, 16, 32, 32])
    inception2 = inception_module(inception1, filters=[128, 128, 192, 32, 96, 64])

    # Batch normalization added after each Inception module
    inception2 = layers.BatchNormalization()(inception2)

    # Global average pooling and dense layers for classification
    gap = layers.GlobalAveragePooling2D()(inception2)
    outputs = layers.Dense(num_classes, activation='softmax')(gap)

    # Create model
    model = Model(inputs=inputs, outputs=outputs)
    return model

**Create and Compile the model**

In [17]:
input_shape = (32, 32, 3)
num_classes = 10  # Number of classes in CIFAR-10 dataset

# Create the model
model = create_inception_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

**Training**

In [18]:
history = model.fit(train_images, train_labels, batch_size=64, epochs=20, validation_data=(test_images, test_labels))

train_accuracy = history.history['accuracy'][-1]
val_accuracy = history.history['val_accuracy'][-1]

print(f"Last training accuracy: {train_accuracy:.4f}")
print(f"Last validation accuracy: {val_accuracy:.4f}")

Epoch 1/20
782/782 [==============================] - 14s 12ms/step - loss: 1.4046 - accuracy: 0.4940 - val_loss: 1.3166 - val_accuracy: 0.5409
Epoch 2/20
782/782 [==============================] - 8s 11ms/step - loss: 1.0563 - accuracy: 0.6267 - val_loss: 1.3374 - val_accuracy: 0.5519
Epoch 3/20
782/782 [==============================] - 8s 10ms/step - loss: 0.9080 - accuracy: 0.6805 - val_loss: 1.2676 - val_accuracy: 0.5949
Epoch 4/20
782/782 [==============================] - 9s 11ms/step - loss: 0.8077 - accuracy: 0.7171 - val_loss: 0.9722 - val_accuracy: 0.6644
Epoch 5/20
782/782 [==============================] - 8s 10ms/step - loss: 0.7250 - accuracy: 0.7464 - val_loss: 0.9531 - val_accuracy: 0.6787
Epoch 6/20
782/782 [==============================] - 8s 10ms/step - loss: 0.6640 - accuracy: 0.7688 - val_loss: 0.9840 - val_accuracy: 0.6648
Epoch 7/20
782/782 [==============================] - 8s 11ms/step - loss: 0.6065 - accuracy: 0.7894 - val_loss: 0.7938 - val_accuracy: 0.729